Read in the data

In [1]:
import pyprind
import pandas as pd
import os

basepath = '../DATA/aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000) # Number of documents
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:57


Shuffle the data and save

In [2]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

df.to_csv('../DATA/movie_data.csv', index=False, encoding='utf-8')

Confirm we saved the file

In [3]:
import pandas as pd

df = pd.read_csv('../DATA/movie_data.csv', encoding='utf-8')
df.head(3)
print(df.shape)

(50000, 2)


## Bag-of-words

Machine learning model cannot work on text. therefore the text has to be converted to 'numerical feature vectors'.

We can use the 'bag-of-words' model:
1. Create a vocabulary of unique tokens (words, etc.) from the entire set of documents
2. Create a feature vectory with the counts of how often each word occurs in a particular document.

In [4]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [5]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [6]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


These values in the feature vectors are called raw term frequencies: tf(t,d) - The number of times a term t occurs in a document d.

Can use 2-gram representation using

In [7]:
count = CountVectorizer(ngram_range=(2,2))
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [8]:
print(count.vocabulary_)

{'the sun': 9, 'sun is': 7, 'is shining': 1, 'the weather': 10, 'weather is': 11, 'is sweet': 2, 'shining the': 6, 'sweet and': 8, 'and one': 0, 'one and': 4, 'one is': 5, 'is two': 3}


In [9]:
print(bag.toarray())

[[0 1 0 0 0 0 0 1 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0 1 1]
 [2 1 1 1 1 1 1 1 1 1 1 1]]


Downweight the frequently common words using term frequency-inverse document frequency (tf-idf)
tf-idf(t,d) = tf(t,d) x idf(t,d)

idf(t,d) = log(n_d / (1 + df(d,t)))

df(d,t) - The number of documents d that contain the term t.

In [10]:
# From the ngram = 1 model
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

[[0.   0.58 0.   0.   0.   0.   0.   0.58 0.   0.58 0.   0.  ]
 [0.   0.   0.58 0.   0.   0.   0.   0.   0.   0.   0.58 0.58]
 [0.57 0.22 0.22 0.28 0.28 0.28 0.28 0.22 0.28 0.22 0.22 0.22]]


pre-process text by removing punctuations marks expect for emoticons

In [11]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [12]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # Remove HTML markup
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text) # Remove emoticons
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', '')) # Remove all non-words characters and convert the text to lower case
    return text

In [13]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [14]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

Apply preprocessor to the movie reviews

In [15]:
df['review'] = df['review'].apply(preprocessor)

Split the text into individuals elements. You can split the text into invidual words by splitting the cleaned documents at its whitespace characters

In [16]:
def tokenizer(text):
    return text.split()

In [17]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

Another useful technique is word stemming which is the process of transforming a word into its root form. The original stemmer was the Porter Stemmer. It does create some fake words e.g. 'thu' which can be corrected using lemmatization (lemmas of the words) but it's computationally expensive and often not worth is

In [18]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [19]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [20]:
from nltk.stem.snowball import SnowballStemmer

snowball = PorterStemmer()

def tokenizer_snowball(text):
    return [snowball.stem(word) for word in text.split()]

In [21]:
tokenizer_snowball('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [22]:
from nltk.stem.lancaster import LancasterStemmer

lancaster = LancasterStemmer()

def tokenizer_lancaster(text):
    return [lancaster.stem(word) for word in text.split()]

In [23]:
tokenizer_lancaster('runners like running and thus they run')

['run', 'lik', 'run', 'and', 'thu', 'they', 'run']

Remove stopwords (e.g. and, is)

In [24]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ray/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
from nltk.corpus import stopwords

stop = stopwords.words('english')

[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

Train a logistic regression model to classify movie reviews.
Do 50% train, 50% split

In [ ]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [ ]:
# Use an updated tokenizer
def tokenizer2(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

Use GridSearchCV to find the optimal parameters using 5-fold stratified cross-validation

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

# tfidf with defaul settings (use_idf=True, smooth_idf=True and norm='l2')
#and
# use_idf=False, norm=None (raw tf)
# lr with L2 and L1 reg and (1, 10, 100) for the inv-reg param C
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               #'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__tokenizer': [tokenizer, tokenizer2],               
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               #'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__tokenizer': [tokenizer, tokenizer2],               
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1) # n_jobs = -1 to use all cores

In [ ]:
gs_lr_tfidf.fit(X_train, y_train)

Print the best parameter set (regular tokenizer without Porter stemming, no stop words and tf-idfs)
lr: L2 reg with C=10

In [ ]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

Using the best model print the average 5-fold cross-validation accuracy on the training set and the classification accuracy on the test dataset

In [ ]:
# .best_score_ is the average k-fold score 
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

# Chose the best fit as the classifier
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

In [ ]:
print(clf.predict(['I love this movie']))
print(np.max(clf.predict_proba(['I love this movie']))*100)

In [ ]:
clf

Use a SGDC to allow for partial fit. Do a GridSearchCV first though

In [30]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21, # make large to avoid hash collisions
                         preprocessor=None, 
                         tokenizer=tokenizer)

#parameters = {'penalty':['l2'],
#              'max_iter':[10000],
#              'n_iter_no_change':[1000]} # Set n_iter_no_change large so it doesn't 'early stop'

# Do 80, 20 split
X_train = df.loc[:40000, 'review'].values
y_train = df.loc[:40000, 'sentiment'].values
X_test = df.loc[10000:, 'review'].values
y_test = df.loc[10000:, 'sentiment'].values

#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html
#SGD_lr = SGDClassifier(loss='log') # los='log' => logistic regression
#clf = GridSearchCV(SGD_lr, parameters, cv=5, verbose=1, n_jobs=-1)
clf = SGDClassifier(loss='log',
                    penalty='l2',
                    max_iter=1000000,
                    #n_iter_no_change=10000,
                    #tol = 0.0000001,
                    n_jobs=-1) 

X_train = vect.transform(X_train)
clf.fit(X_train, y_train)
print('Train Accuracy: %.3f' % clf.score(X_train, y_train))
X_test = vect.transform(X_test)
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Train Accuracy: 0.885
Test Accuracy: 0.882


In [32]:
# pickle me pink
import pickle
import os

# Create movieclassifier folder and a pkl_objects sub-folder
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

# Save stop as stopwords.pkl. wb = write binary
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
# Save clf as classifier.pkl
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print('Best parameter set: %s ' % clf.best_params_)

In [ ]:
# .best_score_ is the average k-fold score 
print('CV Accuracy: %.3f' % clf.score)

X_test = vect.transform(X_test)
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test a Niave Bayes classifier (lr is better so skip this cell...)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__alpha': [1.0, 0.1, 0.01]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__alpha': [1.0, 0.1, 0.01]},
              ]

nb_tfidf = Pipeline([('vect', tfidf),
                     ('clf', MultinomialNB())])

gs_nb_tfidf = GridSearchCV(nb_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1) # n_jobs = -1 to use all cores
                           
gs_nb_tfidf.fit(X_train, y_train)

print('Best parameter set: %s ' % gs_nb_tfidf.best_params_)

print('CV Accuracy: %.3f' % gs_nb_tfidf.best_score_)

#Chose the best fit as the classifier
clf = gs_nb_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

## Serializing fitted sklearn estimators

Use pickle for model persistence. Allow us to serialize and desearialize python object structures to bytecode.

Save the classifier along woth the stop words, Porter Stemmer, and HashingVectorizer as serialized objects to our local disk

In [ ]:
import pickle
import os

# Create movieclassifier folder and a pkl_objects sub-folder
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

# Save stop as stopwords.pkl. wb = write binary
#pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
# Save clf as classifier.pkl
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

## Out-of-core learning

Process the data out-of-core to speed up. Clean text data. Seperate into word tokens while removing stop words

In [ ]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

Read one file at a time

In [ ]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

Run the function on the first file

In [ ]:
next(stream_docs(path='../DATA/movie_data.csv'))

Create a function to get N documents

In [ ]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

Use the HashingVectorizing as can't use CountVectorizer or TfidfVectorizer without the whole dataset in memory

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21, # make large to avoid hash collisions
                         preprocessor=None, 
                         tokenizer=tokenizer)

Use a SGDC classifier

In [ ]:
clf = SGDClassifier(loss='log', random_state=1, max_iter=1) # los='log' => logistic regression
doc_stream = stream_docs(path='../DATA/movie_data.csv')

Out-of-core learning on 45 mini-batches where each batch consists of 1,000 documents

In [ ]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

Use the last 5000 documents to test

In [ ]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Try Word2Vec another time

## Topic modeling

assign topics to unlabelled text documents e.g. newspaper docs -> sports, finance, etc.
This is a clustering task (unsupervised learning)

Use LDA Blei et al (2003).
Use 10 topics

In [ ]:
import pandas as pd

df = pd.read_csv('../DATA/movie_data.csv', encoding='utf-8')
df.head(3)

Use CountVectorizer to create the bag-of-words matrix as input to LDA. Use sklearn's stop words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1, # max doc freq to be 10% to exlude frequent words
                        max_features=5000) # 5000 most frequently occuring words
X = count.fit_transform(df['review'].values)

Fit LDA model for 10 topics

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch') # 'batch' is slower than 'online' (mini-batch)
X_topics = lda.fit_transform(X)

See words importance for each topic. e.g. top 5 words for each topic

In [ ]:
lda.components_.shape

In [ ]:
n_top_words = 5
feature_names = count.get_feature_names()

for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

6 looks like horror movies. Plot first 300 characters of three movies reviews

In [ ]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\nHorror movie #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')